In [1]:
import glob
import cv2
import pandas as pd
from keras import models
from keras import layers
from numpy.random import seed
seed(123)
from tensorflow import set_random_seed
set_random_seed(123)
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import numpy as np
from keras.models import model_from_json
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os, shutil
from sklearn.cluster import KMeans

Using TensorFlow backend.


In [11]:
for im in glob.glob('../early_predictions/images/*/*'):
    if im[-4:] != '.jpg':
        new_filename = im + '.jpg'
        os.rename(im, new_filename)
    else:
        pass

## ResNet50 

In [3]:
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = models.Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights="imagenet"))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
def extract_vector_df(path):
    resnet_feature_list = []
    user_list = []
    img_list = []

    for im in glob.glob(path):
        try: 
            print(im.split('/')[3])
            user_list.append(im.split('/')[3])
            img_list.append(im)
            im = cv2.imread(im)
            im = cv2.resize(im,(224,224))
            img = preprocess_input(np.expand_dims(im.copy(), axis=0))
            resnet_feature = my_new_model.predict(img)
            resnet_feature_np = np.array(resnet_feature)
            resnet_feature_list.append(resnet_feature_np.flatten())
        except:
            print('this one is busted')
    return user_list, img_list, resnet_feature_list

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# # get all the data in the directory split/test (180 images), and reshape them
# data_all = ImageDataGenerator(rescale=1./255).flow_from_directory( 
#         '../early_predictions/images/', 
#         target_size=(224, 224), 
#         batch_size = 2048, 
#         seed = 123)

In [ ]:
# images, labels = next(data_all)

In [ ]:
# my_new_model.predict(images)

In [5]:
users, images, vectors = extract_vector_df('../early_predictions/images/*/*')

In [6]:
vectors

[]

In [7]:
kmeans = KMeans(n_clusters=20, random_state=123).fit(vectors)
print(kmeans.labels_)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [8]:
d = {'user': users, 'label': kmeans.labels_, 'image': images}
df2 = pd.DataFrame(d, columns=['user', 'label', 'image'])
df2.head(20)

NameError: name 'kmeans' is not defined

In [130]:
df2.shape

(3559, 3)

In [132]:
df2.to_csv('resnet_kmeans_20.csv', index_label=None )